# 0. Setup Paths

### Steps 0 is to setup paths needed for training, evaluating and testing of the ML model

### Steps 1 to 7 is to download, train and evaluate the ML model

### To load the model (after setting up the necessary libraries) and test on an image, run Step 0 and jump to Step 8 and 9.

### For RealTime detection with Intel RealSense camera, see the other notebook file named "realtime test.ipynb"

In [1]:
import os

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_resnet50_v1_fpn_640x640' # Current best model
PRETRAINED_MODEL_NAME = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8' # Current best model
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [13]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [27]:
if os.name=='nt':
    !pip install wget
    import wget

In [7]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow\models'...

Updating files:  21% (553/2549)

Updating files:  22% (561/2549)

Updating files:  23% (587/2549)

Updating files:  24% (612/2549)

Updating files:  25% (638/2549)

Updating files:  26% (663/2549)

Updating files:  27% (689/2549)

Updating files:  28% (714/2549)

Updating files:  29% (740/2549)

Updating files:  30% (765/2549)

Updating files:  31% (791/2549)

Updating files:  32% (816/2549)

Updating files:  33% (842/2549)

Updating files:  34% (867/2549)

Updating files:  35% (893/2549)

Updating files:  36% (918/2549)

Updating files:  37% (944/2549)

Updating files:  38% (969/2549)

Updating files:  39% (995/2549)

Updating files:  40% (1020/2549)

Updating files:  41% (1046/2549)

Updating files:  42% (1071/2549)

Updating files:  42% (1076/2549)

Updating files:  43% (1097/2549)

Updating files:  44% (1122/2549)

Updating files:  45% (1148/2549)

Updating files:  46% (1173/2549)

Updating files:  47% (1199/2549)

Updating files:  48% (1224/254

In [8]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

100% [..........................................................................] 1468733 / 1468733        1 file(s) moved.
        1 file(s) copied.
running build
running build_py
creating build
creating build\lib
creating build\lib\object_detection
copying object_detection\eval_util.py -> build\lib\object_detection
copying object_detection\eval_util_test.py -> build\lib\object_detection
copying object_detection\exporter.py -> build\lib\object_detection
copying object_detection\exporter_lib_tf2_test.py -> build\lib\object_detection
copying object_detection\exporter_lib_v2.py -> build\lib\object_detection
copying object_detection\exporter_main_v2.py -> build\lib\object_detection
copying object_detection\exporter_tf1_test.py -> build\lib\object_detection
copying object_detection\export_inference_graph.py -> build\lib\object_detection
copying object_detection\export_tflite_graph_lib_tf2.py -> build\lib\object_detection
copying object_detection\export_tflite_graph_lib_tf2_test.py -> build

error: [Errno 2] No such file or directory: 'build\\bdist.win-amd64\\egg\\object_detection\\dataset_tools\\context_rcnn\\__pycache__\\add_context_to_examples_tf2_test.cpython-38.pyc.1289089416048'



copying object_detection\anchor_generators\grid_anchor_generator_test.py -> build\lib\object_detection\anchor_generators
copying object_detection\anchor_generators\multiple_grid_anchor_generator.py -> build\lib\object_detection\anchor_generators
copying object_detection\anchor_generators\multiple_grid_anchor_generator_test.py -> build\lib\object_detection\anchor_generators
copying object_detection\anchor_generators\multiscale_grid_anchor_generator.py -> build\lib\object_detection\anchor_generators
copying object_detection\anchor_generators\multiscale_grid_anchor_generator_test.py -> build\lib\object_detection\anchor_generators
copying object_detection\anchor_generators\__init__.py -> build\lib\object_detection\anchor_generators
creating build\lib\object_detection\box_coders
copying object_detection\box_coders\faster_rcnn_box_coder.py -> build\lib\object_detection\box_coders
copying object_detection\box_coders\faster_rcnn_box_coder_test.py -> build\lib\object_detection\box_coders
copyi

copying build\lib\object_detection\utils\test_utils.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\test_utils_test.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\tf_version.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\variables_helper.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\variables_helper_tf1_test.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\visualization_utils.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\visualization_utils_test.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\vrd_evaluation.py -> build\bdist.win-amd64\egg\object_detection\utils
copying build\lib\object_detection\utils\vrd_evaluation_test.py -> build\bdist.win-amd64\eg

Obtaining file:///C:/Users/limwu/Downloads/ALEX/Vision/ML_Model_and_Labelling/Youtube_course/TFODCourse-main/Tensorflow/models/research/slim
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)
  Running setup.py develop for slim


In [18]:
!cd Tensorflow/models/research && python -m pip install --use-feature=2020-resolver . # Added this line to install the api

Processing c:\users\limwu\downloads\alex\vision\ml_model_and_labelling\youtube_course\tfodcourse-main\tensorflow\models\research
  Using cached avro-python3-1.10.2.tar.gz (38 kB)


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


  Using cached pycocotools-2.0.2-cp38-cp38-win_amd64.whl
  Using cached lvis-0.5.3-py3-none-any.whl (14 kB)
  Using cached tf_models_official-2.5.0-py2.py3-none-any.whl (1.6 MB)
  Using cached hdfs-2.6.0-py3-none-any.whl (33 kB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached crcmod-1.7.tar.gz (89 kB)
  Using cached avro-python3-1.9.2.1.tar.gz (37 kB)
  Using cached dill-0.3.1.1.tar.gz (151 kB)
  Using cached pydot-1.4.2-py2.py3-none-any.whl (21 kB)
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)
  Using cached py-cpuinfo-8.0.0.tar.gz (99 kB)
  Using cached gin_config-0.4.0-py2.py3-none-any.whl (46 kB)
  Using cached kaggle-1.5.12.tar.gz (58 kB)
  Using cached tensorflow_datasets-4.3.0-py3-none-any.whl (3.9 MB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Using cached sacrebleu-1.5.1-py3-none-any.whl (54 kB)
  Using cached uritemplate-3.0.1-py2.py3-n

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=cb9fcc1d65c0cc11a3e95abe92723e650bf6a32e8544b771f4385113b566d3d2
  Stored in directory: c:\users\limwu\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built object-detection avro-python3 crcmod dill future docopt kaggle py-cpuinfo seqeval promise
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.13.0
    Uninstalling absl-py-0.13.0:
      Successfully uninstalled absl-py-0.13.0


In [ ]:
# Installing Coco API
!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

In [19]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-07-02 11:39:05.331124: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
Running tests under Python 3.8.5: C:\Users\limwu\anaconda3\envs\alex_yt\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-07-02 11:39:10.936064: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2021-07-02 11:39:11.940737: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 119.24GiB/s
2021-07-02 11:39:11.940776: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-07-02 11:39:12.021996: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll

I0702 11:39:22.736844 20392 efficientnet_model.py:147] round_filter input=320 output=352
I0702 11:39:22.859996 20392 efficientnet_model.py:147] round_filter input=1280 output=1408
I0702 11:39:22.885540 20392 efficientnet_model.py:458] Building model efficientnet with params ModelConfig(width_coefficient=1.1, depth_coefficient=1.2, resolution=260, dropout_rate=0.3, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=24, output_filters=40, kernel_size=5, num_repeat=2, expand_ratio=6, strides=(2, 2), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=40, output_filters=80, kernel_size=3, num_repeat=3, expa

In [10]:
!pip install tensorflow-gpu --upgrade # Added gpu for own install

  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached wrapt-1.12.1-cp38-cp38-win_amd64.whl
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached Werkzeug-2.0.1-py3-none-any.whl (288 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached googl

In [13]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib==3.2

Found existing installation: protobuf 3.15.7
Uninstalling protobuf-3.15.7:
  Successfully uninstalled protobuf-3.15.7
Found existing installation: matplotlib 3.4.1
Uninstalling matplotlib-3.4.1:
  Successfully uninstalled matplotlib-3.4.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.28.0 requires crcmod<2.0,>=1.7, which is not installed.
apache-beam 2.28.0 requires dill<0.3.2,>=0.3.1.1, which is not installed.
apache-beam 2.28.0 requires fastavro<2,>=0.21.4, which is not installed.
apache-beam 2.28.0 requires future<1.0.0,>=0.18.2, which is not installed.
apache-beam 2.28.0 requires grpcio<2,>=1.29.0, which is not installed.
apache-beam 2.28.0 requires hdfs<3.0.0,>=2.1.0, which is not installed.
apache-beam 2.28.0 requires httplib2<0.18.0,>=0.8, which is not installed.
apache-beam 2.28.0 requires mock<3.0.0,>=1.0.1, which is not installed.
tensorflow 2.5.0rc0 requires astunparse~=1.6.3, which is not installed.
tensorflow 2.5.0rc0 requires flatbuffers~=1.12.0, which is not installed.
tensorflow 2.5.0rc0 requires gast==0.4.0, which is not installed.
tensorflow 2.5.0rc0 requires go


  Using cached matplotlib-3.2.0-cp37-cp37m-win_amd64.whl (9.2 MB)


In [8]:
import object_detection

In [20]:
!pip list # Check that object-detection is on the list

Package                         Version             Location
------------------------------- ------------------- --------------------------------------------------------------------------------------------------------------------------
absl-py                         0.12.0
anyio                           2.2.0
apache-beam                     2.30.0
argon2-cffi                     20.1.0
astunparse                      1.6.3
async-generator                 1.10
attrs                           21.2.0
avro-python3                    1.9.2.1
Babel                           2.9.1
backcall                        0.2.0
backports.functools-lru-cache   1.6.4
bleach                          3.3.0
brotlipy                        0.7.0
cachetools                      4.2.2
certifi                         2020.6.20
cffi                            1.14.5
chardet                         4.0.0
colorama                        0.4.4
contextlib2                     21.6.0
crcmod                         

# 1.1. Download pre-trained model

In [14]:
import wget
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

  0% [                                                                      ]   1728512 / 204067969

  1% [.                                                                     ]   3645440 / 204067969

100% [......................................................................] 204067969 / 204067969        1 file(s) moved.


x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/checkpoint/
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/checkpoint/checkpoint
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/checkpoint/ckpt-0.index
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/pipeline.config
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/saved_model/
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/saved_model/saved_model.pb
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/saved_model/variables/
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x centernet_resnet50_v1_fpn_512x512_coco17_tpu-8/saved_model/variables/variables.index


# 2. Create Label Map

In [5]:
labels = [{'name':'upstairs', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records

In [7]:
# OPTIONAL IF RUNNING ON COLAB
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
  !tar -zxvf {ARCHIVE_FILES}

In [8]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow\scripts'...


In [7]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow\workspace\annotations\train.record
Successfully created the TFRecord file: Tensorflow\workspace\annotations\test.record


# 4. Copy Model Config to Training Folder

In [15]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

        1 file(s) copied.


# 5. Update Config For Transfer Learning

In [16]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [22]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [23]:
config

{'model': ssd {
   num_classes: 1
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 896
       max_dimension: 896
       pad_to_max_dimension: true
     }
   }
   feature_extractor {
     type: "ssd_efficientnet-b3_bifpn_keras"
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: SWISH
       batch_norm {
         decay: 0.9900000095367432
         scale: true
         epsilon: 0.0010000000474974513
       }
       force_use_bias: true
     }
     bifpn {
       min_level: 3
       max_level: 7
       num_iterations: 6
       num_filters: 160
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 1.0
       x_scale: 1.0
       height_scale: 1.0
       width_scale: 1.0
     }
   }
   matcher {
     argmax_matc

In [19]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [20]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [21]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [7]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [25]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [26]:
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_efficientdet_d3 --pipeline_config_path=Tensorflow\workspace\models\my_efficientdet_d3\pipeline.config --num_train_steps=2000


In [ ]:
!{command}

# 7. Evaluate the Model

In [8]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [9]:
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_efficientdet_d3 --pipeline_config_path=Tensorflow\workspace\models\my_efficientdet_d3\pipeline.config --checkpoint_dir=Tensorflow\workspace\models\my_efficientdet_d3


In [22]:
!{command}

^C


# 8. Load Train Model From Checkpoint

In [5]:
import os
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [6]:
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-17')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Done! Took 5.591118335723877 seconds


# 9. Detect from an Image

In [7]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'],use_display_name=True)

In [19]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'stairs_62.jpg') # Change the last variable to image file name

In [23]:
# start_time = time.time()
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False) # can add line_thickness=8 to change border thickness

cv2.namedWindow('Stairs',cv2.WINDOW_NORMAL)
cv2.resizeWindow('Stairs',(1000,1000))
cv2.imshow('Stairs',image_np_with_detections)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(detections['detection_scores'][0])

# end_time = time.time()
# elapsed_time = end_time - start_time
# print('Done! Took {} seconds'.format(elapsed_time))

0.9195845


## 9.1.1 Cropping and viewing 3D point cloud
http://www.open3d.org/docs/0.9.0/tutorial/Basic/working_with_numpy.html

### Cropping RGB image

In [34]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)
height, width, channels = img.shape
ymin, xmin, ymax, xmax = detections['detection_boxes'][0]
bbox = [int(ymin*height), int(xmin*width), int(ymax*height), int(xmax*width)]
print("Bounding box coordinate of edge lines: [ymin, xmin, ymax, xmax] = " + str(bbox))

# Uncomment below to draw dots on corner of bounding box in the image
# Drawing dots on top left corner of box and bottom right corner of box
# image = cv2.circle(img, center=(bbox[1],bbox[0]), radius=30, color=(0, 0, 255), thickness=-1)
# image = cv2.circle(img, center=(bbox[3],bbox[2]), radius=30, color=(0, 0, 255), thickness=-1)
# cv2.namedWindow('Stairs',cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Stairs',(1000,1000))
# cv2.imshow('Stairs',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Display cropped RGB image
color_crop_img_np = image_np[bbox[0]:bbox[2], bbox[1]:bbox[3]]
uint16_color_crop_img_np = np.asarray(image_np[bbox[0]:bbox[2], bbox[1]:bbox[3]], dtype='uint16')
cv2.imshow("Cropped Image", color_crop_img_np)
cv2.waitKey(0)
cv2.destroyAllWindows()

Bounding box coordinate of edge lines: [ymin, xmin, ymax, xmax] = [280, 70, 472, 633]


### Cropping Depth image

In [35]:
import open3d as o3d
import cv2
# Crop depth image
depth = cv2.imread(DEPTH_IMAGE_PATH, cv2.IMREAD_UNCHANGED)
depth_img_np = np.array(depth)
depth_crop_img_np = depth_img_np[bbox[0]:bbox[2], bbox[1]:bbox[3]]

### Saving cropped files

In [22]:
# Save the cropped images
cv2.imwrite('cropped_color_img3.png', color_crop_img_np)
cv2.imwrite('cropped_depth_img3.png', depth_crop_img_np)

True

### Loading uncropped/cropped files

- Room for improvement: use o3d.io.read_image() to read the images directly from their numpy arrays (post-cropping)
    - It works for depth as it is single dimension, but has issues with the RGB because o3d reads the RGB in reversed dimensions as BGR instead.

In [29]:
# Temporary image path to rgb and depth test images
IMAGE_PATH = os.path.join('Other-files', 'rgb_img.png')
DEPTH_IMAGE_PATH = os.path.join('Other-files', 'depth_img.png')
CROPPED_RGB_PATH = os.path.join('Other-files', 'cropped_color_img3.png')
CROPPED_DEPTH_PATH = os.path.join('Other-files', 'cropped_depth_img3.png')

## 9.1.2 Plane Detection script

ToDo:
- <font color='red'>Add together crop code above into ML detection code, so it can be fed into plane detection code below.</font>

In [30]:
import numpy as np
import cv2 
import open3d as o3d
##import pyrealsense2 as rs

def ReadPlyPoint(fname):
    """ read point from ply
    Args:
        fname (str): path to ply file
    Returns:
        [ndarray]: N x 3 point clouds
    """

    pcd = o3d.io.read_point_cloud(fname)

    return PCDToNumpy(pcd)


def NumpyToPCD(xyz):
    """ convert numpy ndarray to open3D point cloud 
    Args:
        xyz (ndarray): 
    Returns:
        [open3d.geometry.PointCloud]: 
    """

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    return pcd


def PCDToNumpy(pcd):
    """  convert open3D point cloud to numpy ndarray
    Args:
        pcd (open3d.geometry.PointCloud): 
    Returns:
        [ndarray]: 
    """

    return np.asarray(pcd.points)


def RemoveNan(points):
    """ remove nan value of point clouds
    Args:
        points (ndarray): N x 3 point clouds
    Returns:
        [ndarray]: N x 3 point clouds
    """

    return points[~np.isnan(points[:, 0])]


def RemoveNoiseStatistical(pc, nb_neighbors=20, std_ratio=2.0):
    """ remove point clouds noise using statitical noise removal method
    Args:
        pc (ndarray): N x 3 point clouds
        nb_neighbors (int, optional): Defaults to 20.
        std_ratio (float, optional): Defaults to 2.0.
    Returns:
        [ndarray]: N x 3 point clouds
    """

    pcd = NumpyToPCD(pc)
    cl, ind = pcd.remove_statistical_outlier(
        nb_neighbors=nb_neighbors, std_ratio=std_ratio)

    return PCDToNumpy(cl)


def DownSample(pts, voxel_size=0.003):
    """ down sample the point clouds
    Args:
        pts (ndarray): N x 3 input point clouds
        voxel_size (float, optional): voxel size. Defaults to 0.003.
    Returns:
        [ndarray]: 
    """

    p = NumpyToPCD(pts).voxel_down_sample(voxel_size=voxel_size)

    return PCDToNumpy(p)


def PlaneRegression(points, threshold=0.01, init_n=3, iter=1000):
    """ plane regression using ransac
    Args:
        points (ndarray): N x3 point clouds
        threshold (float, optional): distance threshold. Defaults to 0.003.
        init_n (int, optional): Number of initial points to be considered inliers in each iteration
        iter (int, optional): number of iteration. Defaults to 1000.
    Returns:
        [ndarray, List]: 4 x 1 plane equation weights, List of plane point index
    """

    pcd = NumpyToPCD(points)

    w, index = pcd.segment_plane(
        threshold, init_n, iter)

    return w, index


def DrawResult(points, colors):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([pcd])

def DetectMultiPlanes(points, min_ratio=0.05, threshold=0.01, iterations=1000):
    """ Detect multiple planes from given point clouds
    Args:
        points (np.ndarray): 
        min_ratio (float, optional): The minimum left points ratio to end the Detection. Defaults to 0.05.
        threshold (float, optional): RANSAC threshold in (m). Defaults to 0.01.
    Returns:
        [List[tuple(np.ndarray, List)]]: Plane equation and plane point index
    """

    plane_list = []
    N = len(points)
    target = points.copy()
    count = 0

    while count < (1 - min_ratio) * N:
        w, index = PlaneRegression(
            target, threshold=threshold, init_n=5, iter=iterations)
    
        count += len(index)
        plane_list.append((w, target[index]))
        target = np.delete(target, index, axis=0)

    return plane_list


if __name__ == "__main__":
    import random
    
#     depth = o3d.geometry.Image(depth_crop_img_np)
#     color = o3d.geometry.Image(color_crop_img_np)
    depth = o3d.io.read_image(CROPPED_DEPTH_PATH) # Loading cropped depth image
    color = o3d.io.read_image(CROPPED_RGB_PATH)   # Loading cropped RGB image
    
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(color,depth)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, 
            o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
    
    points = PCDToNumpy(pcd)
    # pre-processing
    #points = RemoveNan(points)
#     points = DownSample(points,voxel_size=0.03)
    #points = RemoveNoiseStatistical(points, nb_neighbors=20, std_ratio=0.6)

    #DrawPointCloud(points, color=(0.4, 0.4, 0.4))

    results = DetectMultiPlanes(points, min_ratio=0.01, threshold=0.03, iterations=2000)
    planes = []
    colors = []
    for _, plane in results:

        r = random.random()
        g = random.random()
        b = random.random()

        color = np.zeros((plane.shape[0], plane.shape[1]))
        color[:, 0] = r
        color[:, 1] = g
        color[:, 2] = b

        planes.append(plane)
        colors.append(color)
    
    planes = np.concatenate(planes, axis=0)
    colors = np.concatenate(colors, axis=0)
#     print(planes)
    DrawResult(planes, colors)


# 9.2. Real Time Detections with Intel Realsense

https://dev.intelrealsense.com/docs/tensorflow-with-intel-realsense-cameras
https://github.com/IntelRealSense/librealsense/blob/master/wrappers/tensorflow/example3%20-%20opencv%20deploy.py
https://dev.intelrealsense.com/docs/python2

Few things to check:
1. Downloaded trained model checkpoint/graph files
2. Have libraries installed

# 10. Real Time Detections from your Webcam

In [ ]:
!pip uninstall opencv-python-headless -y

In [13]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

# 10. Freezing the Graph

In [24]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [25]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [26]:
print(command)

python Tensorflow\models\research\object_detection\exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=Tensorflow\workspace\models\my_ssd_resnet50_v1_fpn_640x640\pipeline.config --trained_checkpoint_dir=Tensorflow\workspace\models\my_ssd_resnet50_v1_fpn_640x640 --output_directory=Tensorflow\workspace\models\my_ssd_resnet50_v1_fpn_640x640\export


In [27]:
!{command}

2021-08-05 02:09:29.086177: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-08-05 02:09:34.345002: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2021-08-05 02:09:34.691952: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 119.24GiB/s
2021-08-05 02:09:34.691982: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2021-08-05 02:09:34.699309: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2021-08-05 02:09:34.699337: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublasLt64_11.dll


# 11. Conversion to TFJS

In [19]:
!pip install tensorflowjs

  Using cached tensorflowjs-3.3.0-py3-none-any.whl (63 kB)
  Using cached tensorflow_hub-0.9.0-py2.py3-none-any.whl (103 kB)
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [20]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])

In [21]:
print(command)

tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default Tensorflow\workspace\models\my_ssd_mobnet\export\saved_model Tensorflow\workspace\models\my_ssd_mobnet\tfjsexport


In [22]:
!{command}

Writing weight file Tensorflow\workspace\models\my_ssd_mobnet\tfjsexport\model.json...


2021-04-03 11:54:23.153051: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:54:25.644887: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 11:54:25.645576: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2021-04-03 11:54:25.667969: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2021-04-03 11:54:25.668001: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:54:25.671400: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202

In [ ]:
# Test Code: https://github.com/nicknochnack/RealTimeSignLanguageDetectionwithTFJS

# 12. Conversion to TFLite

In [23]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [24]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])

In [25]:
print(command)

python Tensorflow\models\research\object_detection\export_tflite_graph_tf2.py  --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config --trained_checkpoint_dir=Tensorflow\workspace\models\my_ssd_mobnet --output_directory=Tensorflow\workspace\models\my_ssd_mobnet\tfliteexport


In [26]:
!{command}

2021-04-03 11:55:05.530772: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:08.004889: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 11:55:08.005672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2021-04-03 11:55:08.028532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2021-04-03 11:55:08.028559: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:08.032092: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202

2021-04-03 11:55:14.884114: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      0 
2021-04-03 11:55:14.884117: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1280] 0:   N 
2021-04-03 11:55:14.884182: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 6611 MB memory) -> physical GPU (device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:2b:00.0, compute capability: 7.5)
2021-04-03 11:55:14.884197: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
W0403 11:55:15.360029 15552 save_impl.py:78] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x00000224AC2B1908>, because it is not built.
W0403 11:55:15.925411 15552 save_impl.py:78] Skipping full serialization of Keras layer <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x00000224D6D42B00>, bec

In [27]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [28]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [29]:
print(command)

tflite_convert --saved_model_dir=Tensorflow\workspace\models\my_ssd_mobnet\tfliteexport\saved_model --output_file=Tensorflow\workspace\models\my_ssd_mobnet\tfliteexport\saved_model\detect.tflite --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops


In [30]:
!{command}

2021-04-03 11:55:38.653963: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:41.159460: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-03 11:55:41.160164: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2021-04-03 11:55:41.183623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2021-04-03 11:55:41.183649: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2021-04-03 11:55:41.187402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202

# 13. Zip and Export Models 

In [ ]:
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 14. Calculating IoU


In [9]:
import cv2
import numpy as np

def get_cv2_img_box(image_path):
    img = cv2.imread(image_path)
    image_np = np.array(img)
    height, width, channels = img.shape
    
    # Run TFOD
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
     
    confidence = detections['detection_scores'][0]
    
    ymin, xmin, ymax, xmax = detections['detection_boxes'][0]

    bb2 = {
      "x1": int(xmin*width),
      "x2": int(xmax*width),
      "y1": int(ymin*height),
      "y2": int(ymax*height)
    }
    return (confidence, bb2)

# print(bb2)

In [10]:
import xml.etree.ElementTree as ET

def extract_boxes(filename):        
    # load and parse the file
    tree = ET.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    for box in root.findall('.//bndbox'):
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        boxes = [xmin, ymin, xmax, ymax]
        
    # extract image dimensions
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height

def get_xml_ground_truth(xml_path):
    """
    Extracts ground truth box coordinates from XML file

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    bb1
        
    """
    
    bbs = extract_boxes(xml_path)

    bb1 = {
      "x1": bbs[0][0],
      "x2": bbs[0][2],
      "y1": bbs[0][1],
      "y2": bbs[0][3]
    }
    return bb1

# print(bb1)

In [11]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

# print(get_iou(bb1, bb2))

In [15]:
IOU_THRESH = 0.5
CONF_THRESH = 0.5

directory = os.path.join(paths['IMAGE_PATH'], 'test')
conf_count = 0
iou_count = 0
tp_count = 0
fp_count = 0
count = 0

conf_lst = []
iou_lst = []

for filename in os.listdir(directory):
    if filename.endswith(".xml"): 
        print(os.path.join(directory, filename))
        img_filename = filename[0:-4] + ".jpg"
        print(os.path.join(directory, img_filename))

        bb1 = get_xml_ground_truth(os.path.join(directory, filename))        
        (confidence, bb2) = get_cv2_img_box(os.path.join(directory, img_filename))

        iou = get_iou(bb1,bb2)        
        print("IoU = " + str(iou))
        print("Confidence = " + str(confidence))
        
        conf_lst.append(confidence)
        iou_lst.append(iou)

        if (confidence > CONF_THRESH):
            conf_count = conf_count + 1
            if (iou > IOU_THRESH):
                tp_count = tp_count + 1
            else:
                fp_count = fp_count + 1

        if (iou > IOU_THRESH):
            iou_count = iou_count + 1
            
        count = count + 1
        continue
    else:
        continue

print("Total number of files: " + str(count))
print("Total Confidence > 0.5: " + str(conf_count))
print("Total IoU > 0.5: " + str(iou_count))
print("True positives: " + str(tp_count))
print("False positives: " + str(fp_count))

Tensorflow\workspace\images\test\phone-stairs-wh (11).xml
Tensorflow\workspace\images\test\phone-stairs-wh (11).jpg
IoU = 0.4763080708884408
Confidence = 0.85125047
Tensorflow\workspace\images\test\phone-stairs-wh (9).xml
Tensorflow\workspace\images\test\phone-stairs-wh (9).jpg
IoU = 0.8378297311649417
Confidence = 0.9597059
Tensorflow\workspace\images\test\stairs-sk (21).xml
Tensorflow\workspace\images\test\stairs-sk (21).jpg
IoU = 0.7786088151324302
Confidence = 0.96250993
Tensorflow\workspace\images\test\stairs-sk (37).xml
Tensorflow\workspace\images\test\stairs-sk (37).jpg
IoU = 0.7125322225446847
Confidence = 0.7693552
Tensorflow\workspace\images\test\stairs-sk (42).xml
Tensorflow\workspace\images\test\stairs-sk (42).jpg
IoU = 0.8541485748885351
Confidence = 0.6499599
Tensorflow\workspace\images\test\stairs-sk (5).xml
Tensorflow\workspace\images\test\stairs-sk (5).jpg
IoU = 0.5299744017884249
Confidence = 0.6147483
Tensorflow\workspace\images\test\stairs-sk (53).xml
Tensorflow\wor

In [17]:
print(conf_lst)
print(iou_lst)

tp_cnt = 0
fp_cnt = 0

for conf_t in [0.5,0.6,0.7,0.8]:
    for iou_t in [0.5,0.6,0.7]:
        tp_cnt = 0
        fp_cnt = 0
        for cnt in range(len(conf_lst)):
            if conf_lst[cnt] > conf_t:
                if iou_lst[cnt] > iou_t:
                    tp_cnt = tp_cnt + 1
                else:
                    fp_cnt = fp_cnt + 1
        print("conf_thresh="+str(conf_t)+", iou_thresh="+str(iou_t)+", TP="+str(tp_cnt)+", FP="+str(fp_cnt))

[0.85125047, 0.9597059, 0.96250993, 0.7693552, 0.6499599, 0.6147483, 0.6347212, 0.9949183, 0.9588721, 0.97984135, 0.99932814, 0.65841305, 0.89017946, 0.99696046, 0.9722288, 0.91697776, 0.9821023, 0.099305704, 0.7702619, 0.12538493, 0.87767804, 0.8480871, 0.70330626, 0.9195845, 0.9994281]
[0.4763080708884408, 0.8378297311649417, 0.7786088151324302, 0.7125322225446847, 0.8541485748885351, 0.5299744017884249, 0.8787908665347622, 0.5921310800895446, 0.7156834115262214, 0.7743047977032177, 0.6627582601763145, 0.9430690288025769, 0.8042161984341658, 0.8613712978413329, 0.8814059262118565, 0.678562460254506, 0.6545358247127534, 0.1802741350871201, 0.555730835836237, 0.6067268079352671, 0.6428174240666304, 0.874588590991837, 0.860512237355594, 0.2056196566517623, 0.8091448610736445]
conf_thresh=0.5, iou_thresh=0.5, TP=21, FP=2
conf_thresh=0.5, iou_thresh=0.6, TP=18, FP=5
conf_thresh=0.5, iou_thresh=0.7, TP=14, FP=9
conf_thresh=0.6, iou_thresh=0.5, TP=21, FP=2
conf_thresh=0.6, iou_thresh=0.6, T

# 15. Reading from .bag file

### Also see "Read Bag File.ipynb"

### RGB and Depth colormap display

In [ ]:
#####################################################
##               Read bag from file                ##
#####################################################

import pyrealsense2 as rs
import numpy as np
import cv2
import argparse
import os.path

# # Create object for parsing command-line options
# parser = argparse.ArgumentParser(description="Read recorded bag file and display depth stream in jet colormap.\
#                                 Remember to change the stream fps and format to match the recorded.")
# # Add argument which takes path to a bag file as an input
# parser.add_argument("-i", "--input", type=str, help="test.bag")
# # Parse the command line arguments to an object
# args = parser.parse_args()
# # Safety if no parameter have been given
# if not args.input:
#     print("No input paramater have been given.")
#     print("For help type --help")
#     exit()
# # Check if the given file have bag extension
# if os.path.splitext(args.input)[1] != ".bag":
#     print("The given file is not of correct file format.")
#     print("Only .bag files are accepted")
#     exit()
try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
#     rs.config.enable_device_from_file(config, args.input)
    rs.config.enable_device_from_file(config, "test.bag")

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, rs.format.rgb8, 30)

    # Start streaming from file
    pipeline.start(config)

    # Create opencv window to render image in
    cv2.namedWindow("Intel Realsense", cv2.WINDOW_AUTOSIZE)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    # Streaming loop
    while True:
         # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        # if pressed escape exit program
        if key == 27:
            cv2.destroyAllWindows()
            break

finally:
    pass

### ML detection on just RGB

In [ ]:
#####################################################
##               Read bag from file                ##
#####################################################

import pyrealsense2 as rs
import numpy as np
import cv2
import argparse
import os.path

bagfilename = "20211014_103014.bag"
# bagfilename = "20211014_100902.bag" # elizabeth murdoch building

try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, bagfilename)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, rs.format.rgb8, 30)

    # Start streaming from file
    pipeline.start(config)

    # Create opencv window to render image in
    cv2.namedWindow("Intel Realsense", cv2.WINDOW_AUTOSIZE)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    # Streaming loop
    while True:
         # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        color_image = np.asanyarray(color_frame.get_data())
            
        # Perform object detection on color image from camera stream
        input_tensor = tf.convert_to_tensor(np.expand_dims(color_image, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = color_image.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.85,    # Change this threshold value as needed
                    agnostic_mode=False)    
        
        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', image_np_with_detections)
        key = cv2.waitKey(1)
        # if pressed escape exit program
        if key == 27:
            cv2.destroyAllWindows()
            break

finally:
    pass

### ML detection on RGB with Depth side by side

In [ ]:
#####################################################
##               Read bag from file                ##
#####################################################

import pyrealsense2 as rs
import numpy as np
import cv2
import argparse
import os.path

bagfilename = "20211014_103014.bag"

try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, bagfilename)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, 30)
    config.enable_stream(rs.stream.color, rs.format.rgb8, 30)

    # Start streaming from file
    pipeline.start(config)

    # Create opencv window to render image in
    cv2.namedWindow("Intel Realsense", cv2.WINDOW_AUTOSIZE)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    # Streaming loop
    while True:
         # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        depth_colormap_dim = depth_colormap.shape
            
        # Perform object detection on color image from camera stream
        input_tensor = tf.convert_to_tensor(np.expand_dims(color_image, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = color_image.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.85,    # Change this threshold value as needed
                    agnostic_mode=False)    
            
        color_colormap_dim = image_np_with_detections.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(image_np_with_detections, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))

        
        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        # if pressed escape exit program
        if key == 27:
            cv2.destroyAllWindows()
            break

finally:
    pass